In [1]:
!pip3 install opencorpora-tools
!rm  *.xml
!opencorpora download

Creating annot.opcorpora.xml from http://opencorpora.org/files/export/annot/annot.opcorpora.xml.bz2
..............................................................................................................................
Done.


In [16]:
!rm *.txt

from google.colab import files
files.upload()

Saving dataset_37845_1.txt to dataset_37845_1.txt


{'dataset_37845_1.txt': b'\xd0\x98 \xd0\xb7\xd0\xbb\xd0\xbe\xd0\xb1\xd0\xb0 \xd1\x8d\xd1\x82\xd0\xb0 \xd1\x82\xd0\xb0\xd0\xba \xd0\xb1\xd1\x8b\xd0\xbb\xd0\xb0 \xd0\xbd\xd0\xb5\xd1\x81\xd1\x82\xd0\xb5\xd1\x80\xd0\xbf\xd0\xb8\xd0\xbc\xd0\xb0, \xd1\x87\xd1\x82\xd0\xbe \xd0\xbe\xd0\xbd \xd1\x83\xd0\xb1\xd0\xb8\xd0\xbb \xd0\x9b\xd0\xb0\xd0\xb3\xd0\xb0\xd1\x80\xd0\xbf\xd0\xbe\xd0\xb2\xd0\xb0.\r\n\xd0\x97\xd0\xbd\xd0\xb0\xd0\xb5\xd1\x82, \xd1\x87\xd1\x82\xd0\xbe \xd1\x81\xd0\xb8\xd0\xb6\xd1\x83 \xd0\xb1\xd0\xb5\xd0\xb7 \xd0\xb3\xd1\x80\xd0\xbe\xd1\x88\xd0\xb0, \xd0\xbd\xd0\xb0\xd0\xb4\xd0\xbe \xd1\x81\xd0\xbb\xd0\xb0\xd1\x82\xd1\x8c \xd0\xb2 \xd0\x9c\xd0\xbe\xd1\x81\xd0\xba\xd0\xb2\xd1\x83, \xd0\xb8 \xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb0\xd0\xb5\xd1\x82 \xd0\xb2\xd0\xb8\xd0\xb4, \xd0\xb1\xd1\x83\xd0\xb4\xd1\x82\xd0\xbe \xd0\xb5\xd0\xb3\xd0\xbe \xd0\xbd\xd0\xb5 \xd0\xba\xd0\xb0\xd1\x81\xd0\xb0\xd0\xb5\xd1\x82\xd1\x81\xd1\x8f.\r\n\xd0\x9e\xd0\xb4\xd0\xbd\xd0\xb0 \xd0\xb1\xd0\xbe\xd0\xbb\xd0\xb5\xd0\x

In [3]:
import re
import scipy

import opencorpora

from tqdm import tqdm

from collections import Counter

corpus = opencorpora.load('annot.opcorpora.xml')
corpus

<Corpus revision=4566500 docs:4030 tokens:1989618>

In [0]:
opencorpora_to_study_tokens = {
    'NOUN':'S', # существительные
    
    'ADJF':'A', # прилагательное
    #'ADJS': 'A', # кр. прилагательное

    'VERB':'V', # глаголы
    'PRTF':'V', # причастие
    #'PRTS': 'V', #кр. причастие
    'GRND': 'V', # деепричастие

    'PREP':'PR', # предлоги

    'CONJ':'CONJ', # союзы

    'ADVB':'ADV', # наречие
    'Prnt': 'ADV', # вводные слова
    'PRCL': 'ADV', # частицы
    'INTJ': 'ADV' # междометия
}

error_tag = 'NI' # тег для всего остального, что не вошло в вышеназванные категории

In [17]:
lemma_dictionary = dict()

for doc in tqdm(corpus):

  for token in doc.tokens:
    word = token.source.lower()

    for parsed in token.parses:
      p_o_s = parsed.grammemes[0]
      lemma = parsed.lemma.lower()

      if p_o_s in opencorpora_to_study_tokens:
        p_o_s = opencorpora_to_study_tokens[p_o_s]

        lemma_dictionary[word] = (lemma, p_o_s)

100%|██████████| 4030/4030 [01:47<00:00, 37.48it/s]


In [18]:
with open('dataset_37845_1.txt', 'r') as f:
  dataset = f.read().split('\n')

dataset[:5]

['И злоба эта так была нестерпима, что он убил Лагарпова.',
 'Знает, что сижу без гроша, надо слать в Москву, и делает вид, будто его не касается.',
 'Одна болеет, другая хочет работать, муж не пускает, у третьей ребёнок хилый, в болячках, и не знают, как лечить.',
 'Они нашли работу охранниками в частных компаниях, но зарплата их явно не устраивала.',
 'И когда внизу это замечают, это называется отблеском высшей гармонии.']

In [19]:
result_set = list()

for words in tqdm(dataset):
  result_row = ''
  words = re.findall('\w+', words)
  for word in words:
    lower_word = word.lower()
    dict_word, p_o_s = lemma_dictionary.get(lower_word, (word, error_tag))
    added_part = f"{word}{{{dict_word}={p_o_s}}}"
    result_row = f"{result_row} {added_part}" if result_row else added_part
  result_set.append(result_row)

print()
result_set[:5]

100%|██████████| 201/201 [00:00<00:00, 19235.10it/s]

['И{и=S} злоба{злоба=S} эта{этот=A} так{так=ADV} была{есть=V} нестерпима{нестерпима=NI} что{что=ADV} он{он=NI} убил{убил=V} Лагарпова{Лагарпова=NI}',
 'Знает{знаю=V} что{что=ADV} сижу{сижу=V} без{без=PR} гроша{грош=S} надо{над=PR} слать{слать=NI} в{в=S} Москву{москва=S} и{и=S} делает{делаю=V} вид{вид=S} будто{будто=ADV} его{его=A} не{не=ADV} касается{касаюсь=V}',
 'Одна{один=A} болеет{болею=V} другая{другой=A} хочет{хочу=V} работать{работать=NI} муж{муж=S} не{не=ADV} пускает{пускаю=V} у{у=ADV} третьей{третий=A} ребёнок{ребёнок=S} хилый{хилый=A} в{в=S} болячках{болячках=NI} и{и=S} не{не=ADV} знают{знаю=V} как{как=ADV} лечить{лечить=NI}',
 'Они{Они=NI} нашли{наслал=V} работу{работа=S} охранниками{охранник=S} в{в=S} частных{частный=A} компаниях{компания=S} но{но=ADV} зарплата{зарплата=S} их{их=A} явно{явно=ADV} не{не=ADV} устраивала{устраиваю=V}',
 'И{и=S} когда{когда=ADV} внизу{внизу=PR} это{этот=A} замечают{замечаю=V} это{этот=A} называется{называюсь=V} отблеском{отблеском=NI} высшей{вы

In [20]:
for r in result_set:
  print(r)

И{и=S} злоба{злоба=S} эта{этот=A} так{так=ADV} была{есть=V} нестерпима{нестерпима=NI} что{что=ADV} он{он=NI} убил{убил=V} Лагарпова{Лагарпова=NI}
Знает{знаю=V} что{что=ADV} сижу{сижу=V} без{без=PR} гроша{грош=S} надо{над=PR} слать{слать=NI} в{в=S} Москву{москва=S} и{и=S} делает{делаю=V} вид{вид=S} будто{будто=ADV} его{его=A} не{не=ADV} касается{касаюсь=V}
Одна{один=A} болеет{болею=V} другая{другой=A} хочет{хочу=V} работать{работать=NI} муж{муж=S} не{не=ADV} пускает{пускаю=V} у{у=ADV} третьей{третий=A} ребёнок{ребёнок=S} хилый{хилый=A} в{в=S} болячках{болячках=NI} и{и=S} не{не=ADV} знают{знаю=V} как{как=ADV} лечить{лечить=NI}
Они{Они=NI} нашли{наслал=V} работу{работа=S} охранниками{охранник=S} в{в=S} частных{частный=A} компаниях{компания=S} но{но=ADV} зарплата{зарплата=S} их{их=A} явно{явно=ADV} не{не=ADV} устраивала{устраиваю=V}
И{и=S} когда{когда=ADV} внизу{внизу=PR} это{этот=A} замечают{замечаю=V} это{этот=A} называется{называюсь=V} отблеском{отблеском=NI} высшей{высший=A} гармонии{г

In [0]:
result_name = 'result.txt'
with open(result_name, 'w') as f:
  for r in result_set:
    f.write(r)

files.download( result_name)